In [ ]:
import pandas as pd
import numpy as np
import random as rn
import re
import pickle
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Softmax
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/NLP_sarcastic_totalCleaned.csv')
data.head()

,Question,Answer,QuestionClean,AnswerClean
0,How many Reddit admins does it take to change ...,"None, they like to keep the mods in the dark.",how many reddit admins does it take to change ...,none they like to keep the mods in the dark
1,How many police officers does it take to chang...,"None, because they'll just beat the room for b...",how many police officers does it take to chang...,none because they will just beat the room for ...
2,How many Kings of Spain does it take to screw ...,A: Juan,how many kings of spain does it take to screw ...,a juan
3,How many Alzheimer's patients does it take to ...,TO GET TO THE OTHER SIDE!,how many alzheimer is patients does it take to...,to get to the other side
4,How many people does it take to change a light...,A Brazillion!!! http://imgur.com/c4CJjUd,how many people does it take to change a light...,a brazillion


In [ ]:
!pip install symspellpy

     |████████████████████████████████| 2.6MB 9.2MB/s 


In [ ]:
import pkg_resources
from symspellpy import SymSpell, Verbosity

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
bigram_path = pkg_resources.resource_filename("symspellpy", "frequency_bigramdictionary_en_243_342.txt")

sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
sym_spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)

# utility function
def correct_spellings(text):
    " For a given sentence this function returns a sentence after correctecting spelling of words "
    suggestions = sym_spell.lookup_compound(text, max_edit_distance=2)

    return suggestions[0]._term

In [ ]:

correct_spellings(data['QuestionClean'].iloc[127])

'how many police officers does it take to screw in a light bulb'

In [ ]:
data['QuestionClean']=data['QuestionClean'].astype('str')

In [ ]:
tqdm.pandas()

# Question's spelling correction
data['clean_question'] = data['QuestionClean'].progress_apply(correct_spellings)

100%|██████████| 304486/304486 [13:42<00:00, 370.29it/s]


In [ ]:
data['AnswerClean']=data['AnswerClean'].astype('str')

In [ ]:
data['clean_answer'] = data['AnswerClean'].progress_apply(correct_spellings)

100%|██████████| 304486/304486 [3:40:03<00:00, 23.06it/s]


In [ ]:
  TOSAVE = './preprocessed/'

In [ ]:
data.head(20)

,Question,Answer,QuestionClean,AnswerClean,clean_question,clean_answer
0,How many Reddit admins does it take to change ...,"None, they like to keep the mods in the dark.",how many reddit admins does it take to change ...,none they like to keep the mods in the dark,how many geddit admins does it take to change ...,none they like to keep the mods in the dark
1,How many police officers does it take to chang...,"None, because they'll just beat the room for b...",how many police officers does it take to chang...,none because they will just beat the room for ...,how many police officers does it take to chang...,none because they will just beat the room for ...
2,How many Kings of Spain does it take to screw ...,A: Juan,how many kings of spain does it take to screw ...,a juan,how many kings of spain does it take to screw ...,a juan
3,How many Alzheimer's patients does it take to ...,TO GET TO THE OTHER SIDE!,how many alzheimer is patients does it take to...,to get to the other side,how many alzheimer is patients does it take to...,to get to the other side
4,How many people does it take to change a light...,A Brazillion!!! http://imgur.com/c4CJjUd,how many people does it take to change a light...,a brazillion,how many people does it take to change a light...,a bazillion
5,How many people with dementia does it take to ...,To get to the other side.,how many people with dementia does it take to ...,to get to the other side,how many people with dementia does it take to ...,to get to the other side
6,How many software engineers does it take to ch...,None. It's a hardware problem.,how many software engineers does it take to ch...,none it is a hardware problem,how many software engineers does it take to ch...,none it is a hardware problem
7,How many Honor Guards does it take to screw in...,A: 22 one to screw it in 21 to shoot the bulb.,how many honor guards does it take to screw in...,a one to screw it in to shoot the bulb,how many honor guards does it take to screw in...,a one to screw it in to shoot the bulb
8,How many McDonald's counter girls does it take...,Two. One to change it and one to put some chip...,how many mcdonald is counter girls does it tak...,two one to change it and one to put some chips...,how many mcdonald is counter girls does it tak...,two one to change it and one to put some chips...
9,How many Freuds does it take to screw in a lig...,Two. One to change the lightbulb and another t...,how many freuds does it take to screw in a lig...,two one to change the lightbulb and another to...,how many freud does it take to screw in a ligh...,two one to change the light bulb and another t...


In [ ]:
data.drop(['QuestionClean','AnswerClean'],1,inplace=True)

In [ ]:
data.head()

,Question,Answer,clean_question,clean_answer
0,How many Reddit admins does it take to change ...,"None, they like to keep the mods in the dark.",how many geddit admins does it take to change ...,none they like to keep the mods in the dark
1,How many police officers does it take to chang...,"None, because they'll just beat the room for b...",how many police officers does it take to chang...,none because they will just beat the room for ...
2,How many Kings of Spain does it take to screw ...,A: Juan,how many kings of spain does it take to screw ...,a juan
3,How many Alzheimer's patients does it take to ...,TO GET TO THE OTHER SIDE!,how many alzheimer is patients does it take to...,to get to the other side
4,How many people does it take to change a light...,A Brazillion!!! http://imgur.com/c4CJjUd,how many people does it take to change a light...,a bazillion


In [ ]:
pickle.dump(data, open('data_spell_corrected.pkl', 'wb'))

In [ ]:
df= pickle.load(open('data_spell_corrected.pkl', 'rb'))

In [ ]:
df.shape

(304486, 4)

In [ ]:
from collections import Counter
qsn_cnt = Counter()
for text in df['clean_question'].values:
    for word in text.split():
        qsn_cnt[word] += 1

qsn_dict = dict(qsn_cnt)

len(qsn_dict.keys())

29881

In [ ]:
from collections import Counter
ans_cnt = Counter()
for text in df['clean_answer'].values:
    for word in text.split():
        ans_cnt[word] += 1

ans_dict = dict(ans_cnt)

len(ans_dict.keys())

44541

In [ ]:
x = np.array(list(qsn_dict.values()))
y = np.array(list(ans_dict.values()))

In [ ]:
np.percentile(x, [25, 50, 75])

array([ 2.,  4., 17.])

In [ ]:
np.percentile(y, [25, 50, 75])

array([ 2.,  6., 25.])

In [ ]:
print(np.percentile(x, [99+i/10 for i in range(10)]))

print(np.percentile(y, [99+i/10 for i in range(10)]))

[  923.8   1017.72  1229.76  1383.84  1685.44  2226.2   2693.6   3730.48
  6119.76 13681.92]
[ 2022.6   2247.12  2515.68  2929.06  3614.8   4323.6   5574.84  7647.12
 13016.24 28303.34]


In [ ]:
def remove_rare_words(sent, min_count, cnt_dict):
    """ To remove rare words from texts """
    new_sent = ''
    for word in sent.split():
        try:
            if cnt_dict[word]>min_count:
                new_sent += word+' '
            else:
                new_sent += '<unk>'+' '
        except:
            new_sent += '<unk> '
    return new_sent.strip()

In [ ]:
remove_rare_words("hello subrata", 2, qsn_dict)

'hello <unk>'

In [ ]:
tqdm.pandas()

In [ ]:
# Removing rare words from texts

#data['clean_question'] = data['clean_question'].progress_apply(remove_rare_words, args=(1, qsn_dict))
#data['clean_answer'] = data['clean_answer'].apply(remove_rare_words, args=(2, ans_dict))

In [ ]:
#RESTRUCTURING DATA

In [ ]:
df['clean_answer'] = '<start> ' + df['clean_answer'].astype(str) + ' <end>'

In [ ]:
df['qsn_len'] = df['clean_question'].apply(lambda x: len(x.split()))
df['ans_len'] = df['clean_answer'].apply(lambda x: len(x.split()))

In [ ]:
df.head()

,Question,Answer,clean_question,clean_answer,qsn_len,ans_len
0,How many Reddit admins does it take to change ...,"None, they like to keep the mods in the dark.",how many geddit admins does it take to change ...,<start> none they like to keep the mods in the...,12,12
1,How many police officers does it take to chang...,"None, because they'll just beat the room for b...",how many police officers does it take to chang...,<start> none because they will just beat the r...,12,13
2,How many Kings of Spain does it take to screw ...,A: Juan,how many kings of spain does it take to screw ...,<start> a juan <end>,14,4
3,How many Alzheimer's patients does it take to ...,TO GET TO THE OTHER SIDE!,how many alzheimer is patients does it take to...,<start> to get to the other side <end>,14,8
4,How many people does it take to change a light...,A Brazillion!!! http://imgur.com/c4CJjUd,how many people does it take to change a light...,<start> a bazillion <end>,13,4


In [ ]:
np.percentile(df['qsn_len'].values, [25, 50, 75, 90, 95, 99])

array([ 6.,  9., 11., 14., 17., 26.])

In [ ]:
np.percentile(df['ans_len'].values, [25, 50, 75, 90, 95, 99])

array([  6.,   9.,  17.,  71., 147., 305.])

In [ ]:
# Parameters

# fixing numpy RS
np.random.seed(42)
# fixing tensorflow RS
tf.random.set_seed(32)
# python RS
rn.seed(12)

# Taking maximum words 38
MAXLEN = 39
QSN_VOCAB_SIZE = 29881
ANS_VOCAB_SIZE = 44541

EMBEDDING_SIZE = 300

In [ ]:
df = df[(df['qsn_len']>2) & (df['qsn_len'] <= MAXLEN)]
df = df[(df['ans_len']>2) & (df['ans_len'] <= MAXLEN)]

# Decoder outpu data
df['answer_out'] = df['clean_answer'].apply(lambda x: " ".join(x.split()[1:]))

# Selecting necessary columns
df = df[['clean_question', 'clean_answer', 'answer_out']].copy()
df.rename(columns={'clean_question':'question', 'clean_answer':'answer_inp'}, inplace=True)

df.head()

,question,answer_inp,answer_out
0,how many geddit admins does it take to change ...,<start> none they like to keep the mods in the...,none they like to keep the mods in the dark <end>
1,how many police officers does it take to chang...,<start> none because they will just beat the r...,none because they will just beat the room for ...
2,how many kings of spain does it take to screw ...,<start> a juan <end>,a juan <end>
3,how many alzheimer is patients does it take to...,<start> to get to the other side <end>,to get to the other side <end>
4,how many people does it take to change a light...,<start> a bazillion <end>,a bazillion <end>


In [ ]:
df['answer_inp'].iloc[10]

'<start> a brazilian <end>'

In [ ]:
df['answer_out'].iloc[10]

'a brazilian <end>'

In [ ]:
from sklearn.model_selection import train_test_split
train, validation = train_test_split(df, test_size=0.25, random_state=16)

train.shape, validation.shape

((191831, 3), (63944, 3))

In [ ]:
pickle.dump((train, validation), open('spelldata_train_val.pkl', 'wb'))

In [ ]:
train, validation = pickle.load(open('spelldata_train_val.pkl', 'rb'))

In [ ]:
train.head()

,question,answer_inp,answer_out
252962,what do you call a priest giving a long boring...,<start> an ones theol gist close enough <end>,an ones theol gist close enough <end>
733,how many democrats does it take to change a li...,<start> it only takes one but we were not able...,it only takes one but we were not able to get ...
125874,then there was the olympic sprinter that teste...,<start> he tripped during the dash and won gol...,he tripped during the dash and won gold in the...
208939,thought i could put dolphin in my fish pie,<start> until i noticed i was using all porpoi...,until i noticed i was using all porpoise flour...
156321,top showboat players in football volume,<start> hah shah <end>,hah shah <end>


In [ ]:
enc_tokenizer = Tokenizer(filters='', oov_token='<unk>')
enc_tokenizer.fit_on_texts(train['question'].values)

dec_tokenizer = Tokenizer(filters='', oov_token='<unk>')
dec_tokenizer.fit_on_texts(train['answer_inp'].values)

In [ ]:
pickle.dump((enc_tokenizer, dec_tokenizer), open('spelltokenizer_obj.pkl', 'wb'))

In [ ]:
enc_tokenizer, dec_tokenizer = pickle.load(open('spelltokenizer_obj.pkl', 'rb'))

In [ ]:
vocab_size_qsn = len(enc_tokenizer.word_index.keys())
vocab_size_ans = len(dec_tokenizer.word_index.keys())

print(vocab_size_qsn, vocab_size_ans)

26310 27946


In [ ]:
dec_tokenizer.word_index['<unk>'], dec_tokenizer.word_index['<start>'], dec_tokenizer.word_index['<end>']

(1, 2, 3)

In [ ]:
enc_tokenizer.word_index['<unk>']

1

In [ ]:
train_encoder_inp = enc_tokenizer.texts_to_sequences(train['question'])
train_encoder_inp = pad_sequences(train_encoder_inp, maxlen=MAXLEN, padding='post')

train_decoder_inp = dec_tokenizer.texts_to_sequences(train['answer_inp'])
train_decoder_inp = pad_sequences(train_decoder_inp, maxlen=MAXLEN, padding='post')

train_decoder_out = dec_tokenizer.texts_to_sequences(train['answer_out'])
train_decoder_out = pad_sequences(train_decoder_out, maxlen=MAXLEN, padding='post')

In [ ]:
train_encoder_inp.shape, train_decoder_inp.shape, train_decoder_out.shape

((191831, 39), (191831, 39), (191831, 39))

In [ ]:
val_encoder_inp = enc_tokenizer.texts_to_sequences(validation['question'])
val_encoder_inp = pad_sequences(val_encoder_inp, maxlen=MAXLEN, padding='post')

val_decoder_inp = dec_tokenizer.texts_to_sequences(validation['answer_inp'])
val_decoder_inp = pad_sequences(val_decoder_inp, maxlen=MAXLEN, padding='post')

val_decoder_out = dec_tokenizer.texts_to_sequences(validation['answer_out'])
val_decoder_out = pad_sequences(val_decoder_out, maxlen=MAXLEN, padding='post')

In [ ]:
val_encoder_inp.shape, val_decoder_inp.shape, val_decoder_out.shape

((63944, 39), (63944, 39), (63944, 39))

In [ ]:
pickle.dump((train_encoder_inp, train_decoder_inp, train_decoder_out), open('spelltrain_tokens.pkl', 'wb'))
pickle.dump((val_encoder_inp, val_decoder_inp, val_decoder_out), open('spellval_token.pkl', 'wb'))

In [ ]:
train_encoder_inp, train_decoder_inp, train_decoder_out = pickle.load(open('spelltrain_tokens.pkl', 'rb'))
val_encoder_inp, val_decoder_inp, val_decoder_out = pickle.load(open('spellval_token.pkl', 'rb'))

In [ ]:
train_encoder_inp.shape, train_decoder_inp.shape, train_decoder_out.shape

((191831, 39), (191831, 39), (191831, 39))

In [ ]:
val_encoder_inp.shape, val_decoder_inp.shape, val_decoder_out.shape

((63944, 39), (63944, 39), (63944, 39))

In [ ]:
!pip install fasttext

     |████████████████████████████████| 71kB 5.7MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3095019 sha256=3da14f9d0a5374b5648da7e0985b74242be03cc8dcea3090065344fac1742d1e
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
import fasttext
import fasttext.util

fasttext.util.download_model('en', if_exists='ignore')  # English
ft = fasttext.load_model('cc.en.300.bin')


In [ ]:
ft.get_dimension()

300

In [ ]:
ft.get_word_vector('hello').shape

(300,)

In [ ]:
qsn_embedding_matrix = np.zeros((QSN_VOCAB_SIZE+1, 300))
for word, i in enc_tokenizer.word_index.items():
    embedding_vector = ft.get_word_vector(word)
    if embedding_vector is not None:
        qsn_embedding_matrix[i] = embedding_vector

In [ ]:
qsn_embedding_matrix.shape

(29882, 300)

In [ ]:
ans_embedding_matrix = np.zeros((ANS_VOCAB_SIZE+1, 300))
for word, i in dec_tokenizer.word_index.items():
    embedding_vector = ft.get_word_vector(word)
    if embedding_vector is not None:
        ans_embedding_matrix[i] = embedding_vector

In [ ]:
ans_embedding_matrix.shape

(44542, 300)

In [ ]:
pickle.dump((qsn_embedding_matrix, ans_embedding_matrix), open('spellembedding_matrix.pkl', 'wb'))

In [ ]:
qsn_embedding_matrix1, ans_embedding_matrix1 = pickle.load(open('spellembedding_matrix.pkl', 'rb'))

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.01587803, -0.0225705 , -0.07099195, ...,  0.03178637,
         0.02778677,  0.00359863],
       [ 0.08764305, -0.49590126, -0.04985499, ...,  0.53485096,
        -0.05581036, -0.01659843],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])